**코드에 언급되어 있지 않은 데이터들을 불러오는 것은 최종코드에서 추출한 데이터들을 따로 저장해서 불러왔습니다.**

**사용자의 편의상 가장 간단한 방법이라 생각하였습니다.**

## import module

In [ ]:
!pip install mxnet
!pip install gluonnlp
!pip install sentencepiece
!pip install transformers==3
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

In [ ]:
import pandas as pd
import pickle
import gzip
from tqdm import tqdm

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
from konlpy.tag import Mecab
import warnings

warnings.filterwarnings('ignore')


## Load data & preprocessing

 - Bert용 데이터로 만들기

In [ ]:
data = pd.read_excel(r'D:\KED\submit\본선검증용.xlsx')
data.drop('Unnamed: 0',axis=1,inplace=True)
kedcd = data.KEDCD.unique()
data = data[data.notnull()]
data['BZ_PPOS_ITM_CTT'] = data['BZ_PPOS_ITM_CTT'].str.replace('[^\w\s]', '', regex=True)
data['BZ_PPOS_ITM_CTT'] = data['BZ_PPOS_ITM_CTT'].str.replace("^[0-9]",'')
dic = data.groupby(['KEDCD'])['BZ_PPOS_ITM_CTT'].apply(lambda x: x.values.tolist()).to_dict()
test_df = pd.DataFrame(columns=['KEDCD','BZ_PPOS_ITM_CTT'])
test_df['KEDCD'] = dic.keys()
test_df['BZ_PPOS_ITM_CTT'] = dic.values()
test_df['BZ_PPOS_ITM_CTT'] = test_df['BZ_PPOS_ITM_CTT'].apply(lambda ls: [x for x in ls if str(x) != 'nan'])
test_df['BZ_PPOS_ITM_CTT']=test_df['BZ_PPOS_ITM_CTT'].apply(lambda x: ''.join(x))
test_df2 = test_df.reset_index()
test_df2.drop('KEDCD',axis=1,inplace=True)
test_df2.columns = ['KEDCD','BZ_PPOS_ITM_CTT']
test_df2['KSIC10_BZC_CD'] = 0
test_df2.drop('KEDCD',axis=1,inplace=True)

In [ ]:
test_df2.to_csv('submit/Bert_final_test.txt',sep='\t')

- LSTM용 데이터로 만들기

In [ ]:
with gzip.open('inference_data/X_train_new_se.pickle','rb') as f:
    X_train= pickle.load(f)   # 기존에 학습용 데이터를 불러옴(편의상)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/submit/Bert_final_test.txt',sep='\t')

mecab = Mecab()
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']


X_test = []

text = list(test['BZ_PPOS_ITM_CTT'])

for i in tqdm(range(len(text))):
    temp_X = []
    temp_X = mecab.nouns(text[i]) # 토큰화
    temp_X = [word for word in temp_X if not word in stop_words] # 불용어 제거
    # temp_X = [word for word in temp_X if len(word) > 1]
    X_test.append(temp_X)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
threshold = 11
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
vocab_size = total_cnt - rare_cnt + 1
tokenizer = Tokenizer(num_words = vocab_size) # num_words = vocab_size
tokenizer.fit_on_texts(X_train)

token_X_test = tokenizer.texts_to_sequences(X_test)
max_len = 300 
pad_X_test = pad_sequences(token_X_test, maxlen = max_len)

## Model Inference

- 대분류

In [ ]:
!wget https://raw.githubusercontent.com/winston1214/project/master/KED/bert_test.py

In [ ]:
!python bert_test.py --source Bert_final_test.txt --weights weight/final_big.pt --batch 32 --device 0 --save_csv_name final_big.csv

- 중분류

In [ ]:
!python bert_test.py --source Bert_final_test.txt --weights weight/final_mid.pt --batch 32 --device 0 --save_csv_name final_mid.csv

- 소분류

In [ ]:
!python bert_test.py --source Bert_final_test.txt --weights weight/final_small.pt --batch 32 --device 0 --save_csv_name final_small.csv

- 세세분류

In [ ]:
!python bert_test.py --source Bert_final_test.txt --weights weight/final_sese.pt --batch 32 --device 0 --save_csv_name final_sese.csv

- 세분류

In [ ]:
vocab_size = 18658
max_len = 300
model4 = Sequential()
model4.add(Embedding(vocab_size, 32, input_length=max_len))
model4.add(Dropout(0.3))
model4.add(Conv1D(32, 5, activation='relu'))
model4.add(Conv1D(32, 5, activation='relu'))
model4.add(MaxPooling1D(pool_size=4))
model4.add(LSTM(32))
model4.add(BatchNormalization())
model4.add(Dense(491, activation='softmax'))
model4.summary()
model4 = load_model( '/content/drive/MyDrive/애쓰는 감자/submit/weight/se_model_0520')
pred4 = model4.predict(pad_X_test)

## 결과값 정리

In [ ]:
submit = pd.DataFrame()

In [ ]:
big = pd.read_csv('final_big.csv')
mid = pd.read_csv('final_mid.csv')
small = pd.read_csv('final_small.csv')
sese = pd.read_csv('final_sese.csv')

- 대분류 라벨인코딩 -> 예측 코드로 디코딩

In [ ]:
import string
dic = {}
for k,v in zip([i for i in range(21)],string.ascii_uppercase):
    dic[k] = v
submit['KEDCD'] = test_df2['KEDCD'].astype(str)
submit['pred_big'] = big['pred'].astype(int).map(dic)

- 중분류 라벨인코딩 -> 예측 코드로 디코딩

In [ ]:
train_mid = pd.read_csv('preprocessing_data/train_data_mid.csv',dtype={'mid':str})
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_mid['mid'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mid['label'] = mid['pred'].astype(int).map(mapping)
submit['pred_mid'] = mid['label']

- 소분류 라벨인코딩 -> 예측 코드로 디코딩

In [ ]:
train_small = pd.read_csv('preprocessing_data/train_data_small.csv',dtype={'small':str})
encoder = LabelEncoder() # encoding
encoder.fit(train_small['small'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_)) # decoding
small['label'] = small['pred'].astype(int).map(mapping)  # decoding mapping
submit['pred_small'] = small['label']

- 세세분류 라벨인코딩 -> 예측 코드로 디코딩

In [ ]:
train_sese = pd.read_csv('preprocessing_data/train_data_sese.csv',dtype={'sese':str})
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(train_sese['sese'])
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
sese['label'] = sese['pred'].astype(int).map(mapping)
submit['sese'] = sese['label']

- 세분류 OneHotEncoding -> 예측코드로 디코딩

In [ ]:
train_se = pd.read_csv('train_data_se.csv',dtype={'KEDCD':str,'BZ_PPOS_ITM_CTT':str,'KSIC10_BZC_CD':str,'mid':str,'Big':str,'small':str,'se':str})
ls = train_se['se'].unique().tolist()
final_ls = []
for i in ls:
    if len(i)==4:
        final_ls.append(i)
final_ls = sorted(final_ls)
pred_se_test = []
for i in pred4:
    pred_idx = np.argmax(i)
    pred_se_test.append(final_ls[pred_idx])
submit['pred_se'] = pred_se_test

## 최종 예측 코드 도출

In [ ]:
submit.columns = ['KEDCD','big','mid','small','sese','se']

final_mid = []
final_small = []
# final_se = []
for i in range(len(submit)):
    # 중분류
    ls = []
    mid = submit.iloc[i]['mid']
    small = submit.iloc[i]['small']
    se = submit.iloc[i]['se']
    sese = submit.iloc[i]['sese']
    ls.append(mid)
    ls.append(small[:-1])
    ls.append(se[:-2])
    ls.append(sese[:-3])
    counting = pd.Series(ls).value_counts()
    if counting.values[0] == 1:
        final_mid.append(mid)
    elif counting.values[0] == 2:
        if counting.values[0] == 2:
            final_mid.append(mid)
    else:
        final_mid.append(counting.index[0])
    # 소분류
    ls2 = []
    ls2.append(small[-1])
    ls2.append(se[-2])
    ls2.append(sese[-3])
    counting2 = pd.Series(ls2).value_counts()
    if counting2.values[0] == 1:
        final_small.append(small[-1])
    else:
        final_small.append(counting2.index[0])

In [ ]:
submit['final_mid'] = final_mid
submit['final_small'] = final_small
submit['pred'] = submit['big']+submit['final_mid'] + submit['final_small'].str[-1] + submit['se'].str[-1] + submit['sese'].str[-1]

In [ ]:
final_submit = pd.read_excel('[공모전]팀이름_본선.xlsx',dtype={'KEDCD':str})
final_submit.info()
df = pd.merge(final_submit,submit,on='KEDCD')

In [ ]:
df.to_csv('submit/[공모전]본선_애쓰는감자.xlsx',index=False)